In [ ]:
import pandas as pd
import pickle
import json
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import dtw
import os

In [ ]:
from xgboost import XGBRegressor
import itertools
from sklearn.model_selection import GridSearchCV
import pickle
import cloudpickle
import copy
import scipy.stats as stats
from datetime import datetime, timedelta, date
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

### Helper Functions

In [ ]:
'''
Metrics
'''

def myerrsq(x,y):
    return((x-y)**2)

### s2 predictions, s1 ground truth
def dtw_(s1, s2):
    window=2
    
    s1= pd.DataFrame(s1)
    s2 = pd.DataFrame(s2)
    
    z1=(s1-s1.mean())/(s1.std(ddof=0).apply(lambda m: (m if m > 0.0 else 1.0)))
    z2=(s2-s2.mean())/(s2.std(ddof=0).apply(lambda m: (m if m > 0.0 else 1.0)))

    ### first value simulation second value GT
    dtw_metric = np.sqrt(dtw.dtw(z2[0], z1[0], dist_method=myerrsq, window_type='slantedband',
                               window_args={'window_size':window}).normalizedDistance)
    
    return dtw_metric

def ae(v1,v2):
    v1=np.array(v1)
    v2 = np.array(v2)
    return np.abs(v1 - v2)

# Scale-Free Absolute Error
def sfae(v1,v2):
    
    v1=np.array(v1)
    v2 = np.array(v2)
    
    return ae(v1, v2) / np.mean(v1)

def MAD_mean_ratio(v1, v2):
    """
    MAD/mean ratio
    """
    return np.mean(sfae(v1, v2))

def normed_rmse(v1,v2):
    v1=np.cumsum(v1)
    v2=np.cumsum(v2)
    v1=v1/np.max(v1)
    v2=v2/np.max(v2)
    
    result = v1-v2
    result = (result ** 2).mean()
    result = np.sqrt(result)
    return result

def rmse(v1,v2):
    result = np.array(v1)-np.array(v2)
    result = (result ** 2).mean()
    result = np.sqrt(result)
    return result

def ape(v1,v2):
    v1=np.sum(v1)
    v2=np.sum(v2)
    result = np.abs(float(v1) - float(v2))
    result = 100.0 * result / np.abs(float(v1))
    return result

def smape(A, F):
    A=np.array(A)
    F=np.array(F)
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [ ]:
def eval_predictions(model_id,gt_data, sim_data, narrative_list=[]):
    '''
    Description: Run predictions for LSTM models
    
    Input:
    model_id: 
    gt_data: ground truth count dict
    sim_data: simulation count dict
    target: target dataframe
    narrative_list: list of info ids
    '''
    Gperformance_data=[]

    for Tnarrative in narrative_list:
        Tnarrative = Tnarrative.replace("informationID_", "")
        sim_y=gt_data[Tnarrative]
        y_hat=sim_data[Tnarrative]
        
        print("Test on narrative: %s"%Tnarrative)
        ape_value=ape(sim_y,y_hat)
        print("APE: ",ape_value)

        rmse_value=rmse(sim_y,y_hat)
        print("RMSE: ",rmse_value)

        nrmse_value=normed_rmse(sim_y,y_hat)
        print("NRMSE: ",nrmse_value)

        smape_value=smape(sim_y,y_hat)
        print("SMAPE: ",smape_value)
        
        sfae_value=MAD_mean_ratio(sim_y,y_hat)
        print("SFAE: ",sfae_value)
        
        dtw_value = dtw_(sim_y, y_hat)
        print("DTW :", dtw_value)

        Gperformance_data.append([Tnarrative,ape_value,rmse_value,nrmse_value,smape_value,sfae_value,dtw_value,model_id])

    Gperformance_data=pd.DataFrame(Gperformance_data,columns=['informationID','APE','RMSE','NRMSE','SMAPE','SFAE','DTW','MODEL'])
    
    return Gperformance_data

### Returns validation performance, simulation performance, validation data and simulation data
def helper_performance(paths = [], start_sim_period="", end_sim_period="", evaluation=True):
    ### Gperformance plots.
    performance_data=[]
    performance_data_eval=[]
    sim_data={}
    val_data={}

    ### Iterate over all model paths in a list to load and compute performance metrics (including internal model)
    for path in paths:
        xmodel_name_array=path.split("/")[-1].split("_")
        xmodel_name = xmodel_name_array[-2]
        window = xmodel_name_array[-3]
        xmodel_name_ = xmodel_name + "_" + window
        
        print(xmodel_name_array)
        if len(xmodel_name_array)>3:
            if (xmodel_name_array[1]==start_sim_period) & (xmodel_name_array[2]==end_sim_period):
                print("Into the blender: ",path)
                ### Read validation performance
                Gperformance=pd.read_pickle(path+'/Gperformance_validation.pkl.gz')
                Gperformance['MODEL_ID']=xmodel_name_
                Gperformance["MODEL"] = xmodel_name
                Gperformance["FILE_NAME"] = path
                if evaluation:
                    Gperformance_eval=pd.read_pickle(path+'/Gperformance_simulation.pkl.gz')
                    Gperformance_eval['MODEL_ID']=xmodel_name_
                    Gperformance_eval["MODEL"] = xmodel_name
                    Gperformance_eval["FILE_NAME"] = path

                with open(path+'/simulations_data.pkl.gz', 'rb') as fd:
                    sim_data_=pickle.load(fd)
                    sim_data.setdefault(xmodel_name_,sim_data_)

                ### Read validation data predictions
                with open(path+'/validation_data.pkl.gz', 'rb') as fd:
                    val_data_=pickle.load(fd)
                    val_data.setdefault(xmodel_name_,val_data_)

                ### read ground truth simulation data
                if evaluation:
                    with open(path+'/gt_data_simulations.pkl.gz', 'rb') as fd:
                        gt_data=pickle.load(fd)

                    ### read ground truth validation data
                    with open(path+'/gt_data_validation.pkl.gz', 'rb') as fd:
                        gt_data_val=pickle.load(fd)

                ### add new results to df and add to list
                if evaluation:
                    performance_data_eval.append(Gperformance_eval)
                performance_data.append(Gperformance)

    performance_data=pd.concat(performance_data, ignore_index=True)
    if evaluation:
        performance_data_eval=pd.concat(performance_data_eval, ignore_index=True)
        return performance_data, performance_data_eval, val_data, sim_data
    else:
        return performance_data, val_data, sim_data
    
def ensemble_all_average(paths, gt_path="",model_id="", evaluation=True):
    sim_ensemble = {}
    
    if evaluation:
        with open(gt_path+'/gt_data_simulations.pkl.gz', 'rb') as fd:
                gt=pickle.load(fd)
    
    for path in paths:
        
        with open(path+'/simulations_data.pkl.gz', 'rb') as fd:
            sim_data=pickle.load(fd)
            
        for k,v in sim_data.items():
            if k in sim_ensemble:
                arr = sim_ensemble[k]
                arr.append(list(v))
                sim_ensemble[k] = arr
            else:
                sim_ensemble[k] = [list(v)]
        
    sim_ensemble_final = {}
    for k, v in sim_ensemble.items():

        v_np = np.array(v)
        sim_ensemble_final[k] = np.round(np.mean(v, axis=0))
    
    infoids = list(sim_ensemble_final.keys())
    
    if evaluation:
        # Evaluate simulations
        Gperformance = eval_predictions(model_id, gt, sim_ensemble_final, infoids)
    
        return Gperformance, sim_ensemble_final
    else:
        return sim_ensemble_final
    
def run_blender(val_perf, sim_data, sim_perf="", metric="RMSE", evaluation=True, model_id=""):
    
    ### Get minimum in validation based on metric
    idx = val_perf.groupby(['informationID'])[metric].transform(min) == val_perf[metric]
    val_perf_blended=val_perf[idx]
    val_perf_blended=val_perf_blended.drop_duplicates().reset_index(drop=True)
    
    if evaluation:
        
        sim_perf['unique_id'] = sim_perf['MODEL_ID']+"_"+sim_perf['informationID']
        val_perf_blended['unique_id'] = val_perf_blended['MODEL_ID']+"_"+val_perf_blended['informationID']
        xmodel_ids = list(val_perf_blended['unique_id'].unique())
        sim_perf = sim_perf.loc[sim_perf['unique_id'].isin(xmodel_ids)].reset_index(drop=True)
        sim_perf.drop(columns=['unique_id'], inplace=True)
        
    sim_data_blended={}
    def getBlendedSimulation(row):
        sim_array=sim_data[row['MODEL_ID']][row['informationID']]
        print(row['MODEL_ID'],row['informationID'],sim_array)
        sim_data_blended.setdefault(row['informationID'],sim_array)
    print("\n\n Blended simulation output ---------------")    
    T=val_perf_blended.apply(getBlendedSimulation,axis=1)
    
    if evaluation:
        sim_perf["MODEL"] = model_id
        return val_perf_blended, sim_perf, sim_data_blended
    else:
        return val_perf_blended, sim_data_blended

In [ ]:
### Returns validation performance, simulation performance, validation data and simulation data
def get_validation_data(paths = [], start_sim_period="", end_sim_period="", simulation=True):
    ### Gperformance plots.
    sim_data={}
    val_data={}
    gt_data_val = {}
    gt_data = 0

    ### Iterate over all model paths in a list to load and compute performance metrics (including internal model)
    for path in paths:
        xmodel_name_array=path.split("/")[-1].split("_")
        xmodel_name = xmodel_name_array[-2]
        window = xmodel_name_array[-3]
        xmodel_name_ = xmodel_name + "_" + window
        
        print(xmodel_name_array)
        if len(xmodel_name_array)>3:
            if (xmodel_name_array[1]==start_sim_period) & (xmodel_name_array[2]==end_sim_period):
                print("For the metalearner: ",path)

                with open(path+'/simulations_data.pkl.gz', 'rb') as fd:
                    sim_data_=pickle.load(fd)
                    sim_data.setdefault(xmodel_name_,sim_data_)

                ### Read validation data predictions
                with open(path+'/validation_data.pkl.gz', 'rb') as fd:
                    val_data_=pickle.load(fd)
                    val_data.setdefault(xmodel_name_,val_data_)

                if simulation:
                    gt_data = 0
                else:
                    ### read ground truth simulation data
                    with open(path+'/gt_data_simulations.pkl.gz', 'rb') as fd:
                        gt_data=pickle.load(fd)

                ### read ground truth validation data
                with open(path+'/gt_data_validation.pkl.gz', 'rb') as fd:
                    gt_data_val=pickle.load(fd)

    return val_data, gt_data_val, sim_data, gt_data

### Test boolean is True if we want both input and target. Otherwise only input 
def get_input_target(input_data, target=[], infoids=[], test=False, normalize=False):
    result_input = []
    result_target = []
    for infoid in infoids:

        sample = []

        models = sorted(list(input_data.keys()))

        ### create one-hot encoding vector
        Tindex=infoids.index(infoid)
        narrative_binary=np.zeros(len(infoids))
        narrative_binary[Tindex]=1

        for model in models:
            s = input_data[model][infoid]
            sample.append(s)

        sim_days = len(s)
        ### Generate encodings equal to number of samples (number of days)
        narrative_binary=np.tile(narrative_binary,(sim_days,1))
        
        ### Transpose matrix and concat one-hot encodings per frame
        sample = np.array(sample)
        sample = sample.transpose()
#         print(sample)

        sample = np.concatenate([sample, narrative_binary], axis=1)
#         print(sample.shape)

        result_input.extend(sample)
        
        if not test:
            t = np.array(target[infoid])
            result_target.extend(t)
    
    data_X = np.array(result_input)
    data_X = data_X.astype(np.float64)
    
    number_features_to_scale = data_X.shape[-1] - len(infoids)
    if normalize:
        data_X[:,:number_features_to_scale] = np.log1p(data_X[:,:number_features_to_scale])
    
    if test:
        return data_X
    else:
        data_y = np.array(result_target)
        data_y = data_y.astype(np.float64)
        if normalize:
            data_y = np.log1p(data_y)
        return data_X, data_y
    
def run_predictions(test_X, model, infoids=[], normalize=False):
    
    n_infoids = len(infoids)
    sim_X = np.split(test_X, n_infoids)
    
    ml_preds = {}
    
    for infoid in infoids:
    
        Tindex=infoids.index(infoid)
        sample = sim_X[Tindex]

        y_hat = model.predict(sample)
        
        if normalize:
            y_hat = np.round(np.expm1(y_hat))
        else:
            y_hat = np.round(y_hat)
#             y_hat[y_hat<0] = 0
            y_hat = np.absolute(y_hat)

        ml_preds[infoid] = y_hat
    
    return ml_preds

### Load Models

In [ ]:
start_period = "2020-11-09"
end_period = "2020-11-29"

platform = "platform_name"

task = "total_shares"

domain = "domain_name"

model_type = "local_topics"

save_shares_path = "../MCAS/ml_output/{2}/{0}/{3}/{1}/".format(platform, task, domain, model_type)

gt_path_shares = glob.glob("../MCAS/ml_output/{2}/{0}/{3}/{1}/LSTM*".format(platform, task, domain, model_type))[0]

paths_week_shares = glob.glob("../MCAS/ml_output/{2}/{0}/{3}/{1}/LSTM*".format(platform, task, domain, model_type))

In [ ]:
val_perf, val_data, sim_data = helper_performance(paths=paths_week_shares, start_sim_period=start_period,
                                                                end_sim_period=end_period, evaluation=False)

### Full Ensemble

In [ ]:
ens_data = ensemble_all_average(paths_week_shares, gt_path=gt_path_shares, model_id="MCAS-Ensemble", evaluation=False)

In [ ]:
ens_data

In [ ]:
try:
    os.mkdir(save_shares_path+"MCAS-ENSEMBLE/")
except OSError as error:
    print(error)  

with open(save_shares_path+"MCAS-ENSEMBLE/simulations_data.pkl.gz",'wb') as f:
    pickle.dump(ens_data,f)

### Blended-Exog

In [ ]:
metric="RMSE"
val_perf_ = val_perf.query("MODEL!='Internal'").reset_index(drop=True)
val_perf_blend,sim_blend_data = run_blender(val_perf=val_perf_, sim_data=sim_data,
                                                             sim_perf="", metric=metric, evaluation=False,model_id="MCAS-Blended-Exog")

In [ ]:
try:
    os.mkdir(save_shares_path+"MCAS-BLENDED-EXOG/")
except OSError as error:
    print(error)  

with open(save_shares_path+"MCAS-BLENDED-EXOG/simulations_data.pkl.gz",'wb') as f:
    pickle.dump(sim_blend_data,f)
    
val_perf_blend.to_pickle(save_shares_path+"MCAS-BLENDED-EXOG/Gperformance_validation.pkl.gz")

### Blended-Int

In [ ]:
metric = "RMSE"
val_perf_ = val_perf.query("MODEL=='Internal'").reset_index(drop=True)
val_perf_blend_int,sim_blend_data_int = run_blender(val_perf=val_perf_, sim_data=sim_data,
                                                                         sim_perf="", metric=metric, evaluation=False,model_id="MCAS-Blended-Int")

In [ ]:
import os

try:
    os.mkdir(save_shares_path+"MCAS-BLENDED-INTERNAL/")
except OSError as error:
    print(error)  

with open(save_shares_path+"MCAS-BLENDED-INTERNAL/simulations_data.pkl.gz",'wb') as f:
    pickle.dump(sim_blend_data_int,f)
    
val_perf_blend_int.to_pickle(save_shares_path+"MCAS-BLENDED-INTERNAL/Gperformance_validation.pkl.gz")

### Meta Ensemble

In [ ]:
start_sim = start_period
end_sim = end_period

prev_domain = ""

### Here we need to load up any previous data_X and data_Y only for retraining purposes
prev_path = "/data/CP5_MCAS/MCAS/ml_output/{0}/{1}/{2}/{3}/MCAS-META-ENSEMBLE/".format(prev_domain, platform, model_type,task)

In [ ]:
val_data, val_gt, sim_data, sim_gt = get_validation_data(paths_week_shares, start_sim_period=start_period, end_sim_period=end_period, simulation=True)

In [ ]:
infoids = sorted(list(val_gt.keys()))

In [ ]:
data_X, data_y = get_input_target(val_data, val_gt, infoids=infoids, test=False, normalize=True)

In [ ]:
data_X.shape, data_y.shape

In [ ]:
test_X = get_input_target(sim_data, infoids=infoids, test=True, normalize=True)

In [ ]:
test_X.shape

In [ ]:
try:
    with open(prev_path+"data_x.pkl", "rb") as f:
        old_data_X = pickle.load(f)
        
    with open(prev_path+"data_y.pkl", "rb") as f:
        old_data_y = pickle.load(f)
    print(old_data_X.shape, old_data_y.shape)
    data_X = np.concatenate((old_data_X,data_X),axis=0)
    data_y = np.concatenate((old_data_y,data_y),axis=0)
    print(data_X.shape, data_y.shape)
except:
    print("No previous input/output files")

In [ ]:
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .07, 0.1], #so called `eta` value
              'max_depth': [3, 5, 6, 7, 10],
              'min_child_weight': [1, 2, 3, 4],
              'subsample': [0.7, 0.8, 1.0],
              'colsample_bytree': [0.5, 0.7, 0.8, 1.0],
              'n_estimators': [500]}

In [ ]:
model = XGBRegressor()

In [ ]:
xgb_grid = GridSearchCV(model,
                        parameters,
                        cv = 5,
                        n_jobs = 5,
                        verbose=True)
xgb_grid.fit(data_X,data_y)

In [ ]:
print(xgb_grid.best_params_)
new_params = xgb_grid.best_params_

In [ ]:
model = XGBRegressor(**new_params)
model.fit(data_X, data_y)

In [ ]:
ml_preds = run_predictions(test_X, model=model, infoids=infoids, normalize=True)

In [ ]:
ml_preds

In [ ]:
try:
    os.mkdir(save_shares_path+"MCAS-META-ENSEMBLE/")
except OSError as error:
    print(error)  

In [ ]:
import pickle

with open(save_shares_path+"MCAS-META-ENSEMBLE/best_model_params.pkl", 'wb') as handle:
    pickle.dump(new_params, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(save_shares_path+"MCAS-META-ENSEMBLE/simulations_data.pkl.gz",'wb') as f:
    pickle.dump(ml_preds,f)

In [ ]:
with open(save_shares_path+'MCAS-META-ENSEMBLE/model.pkl','wb') as f:
    pickle.dump(model,f)

In [ ]:
with open(save_shares_path+'MCAS-META-ENSEMBLE/data_x.pkl','wb') as f:
    pickle.dump(data_X,f)
    
with open(save_shares_path+'MCAS-META-ENSEMBLE/data_y.pkl','wb') as f:
    pickle.dump(data_y,f)